In [ ]:
# INTERACTIVE PLOTS (only if you code locally, unfortunately)
# This cell can take some minutes to run
# You should only need to run this cell once and not everytime you open the script
# In case the plotting of the data below does not work because of the error "Error displaying widget: model not found"
# you'll probably have to restart your kernel once more.

# IMPORTANT: After running this cell, restart your kernel
# uncomment the next two lines for interactive plots
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-leaflet
# !pip install install nodejs

In [ ]:
import matplotlib.pyplot as plt

# You may change the mhealth_activity module but your algorithm must support the original version
from mhealth_activity import Recording, Trace, Activity, WatchLocation, Path

# For interactive plots, uncomment the following line
# %matplotlib widget

# Reading sensor data from PKL file

Create an instance of the <code>Recording</code> class that manages the data from a file

In [ ]:
recording = Recording('/kaggle/input/24-exercise2/data/train/train_trace_013.pkl')

# The data is stored inside a dictionary (access: recording.data["key"])
print("\r\nAvailable data traces:")
print(list(recording.data.keys()))

if not(recording.labels is None):
    print("\r\nLabels: ")
    print(recording.labels)

print(f"\r\nRecording:")
for key in list(recording.data.keys()):
    print(f'{key}: {recording.data[key]}')

In [ ]:
print("ACTIVITIES")
for activity in Activity:
    if activity.value in recording.labels["activities"]:
        print(f"{activity.name} is contained in the recording, thus '{activity.name.lower()}': True")
    else:
        print(f"{activity.name} is NOT contained in the recording, thus '{activity.name.lower()}': False")

print("\r\nWATCH LOCATION")
print(f"The LilyGo was worn on the {WatchLocation(recording.labels['watch_loc']).name.lower()}.")

print("\r\nPATH IDX")
print(f"The participant completed {Path(recording.labels['path_idx']).name.lower()}.")

print("\r\nSTEP COUNT")
if "step_count" in recording.labels:
    print(f"The participant performed {recording.labels['step_count']} steps.")
else:
    print("The step count is not available.")


## Working with timeseries
Every value in this dictionary holds a <code>Trace</code> object to manage the data from one sensor

In [ ]:
recording = Recording('/kaggle/input/24-exercise2/data/example/medium_walk.pkl')  # Second example file: data/example/short_walk.pkl

# X-axis of the LILYGO accelerometer
ax = recording.data['ax']

print(f"Name of sensor: '{ax.title}'")
print(f"Sample rate: {int(ax.samplerate)} Hz")
print(f"Recording length: {ax.total_time} seconds")
print(f"Timestamp of recording: {ax.raw_timestamps[0][1]}")
print("\nGetting a time series:")
print(" Time           x-Accel")
for x, t in list(zip(ax.timestamps, ax.values))[:20]:
    print(f"{x: .3f}s \t{t:.5f}g")

# Plotting

In [ ]:
# Display a simple plot
recording.plot(['ax'])

In [ ]:
# Plot multiple axes at the same time
# It's also possible to give titles to the y-axis and plots
recording.plot(['ax', 'ay', 'az'], ylabels=['acceleration [g]'], labels=['Acc X', 'Acc Y', 'Acc Z'])

In [ ]:
# Plot multiple sensors and multiple axes at the same time
recording.plot([['ax', 'ay', 'az'], ['gx', 'gy', 'gz'], ['mx', 'my', 'mz'], ['altitude'], ['speed']], 
             ylabels=['acceleration [g]', 'rot. vel. [deg/s]', 'field strength [µT]', 'altitude [m]', 'speed [m/s]'],
             labels=[['Acc X', 'Acc Y', 'Acc Z'], ['Gyro X', 'Gyro Y', 'Gyro Z'], ['Mag X', 'Mag Y', 'Mag Z'], ['Altitude'], ['Speed']])

In [ ]:
# How to Zoom In: use start_s and end_s to specify the time range in seconds
# start_s: start of the range you want to plot (in seconds since the start of the trace)
# end_s: end of the range you want to plot (in seconds since the start of the trace)
recording.plot([['ax', 'ay', 'az'], ['gx', 'gy', 'gz'], ['mx', 'my', 'mz'], ['altitude'], ['speed']], 
             ylabels=['acceleration [g]', 'rot. vel. [deg/s]', 'field strength [µT]', 'altitude [m]', 'speed [m/s]'],
             labels=[['Acc X', 'Acc Y', 'Acc Z'], ['Gyro X', 'Gyro Y', 'Gyro Z'], ['Mag X', 'Mag Y', 'Mag Z'], ['Altitude'], ['Speed']],
               start_s=10, end_s=70)

# Display GPS data

In [ ]:
# Recording must contain gps data
assert('latitude' in recording.data and 'longitude' in recording.data)

plot_map_zurich = True

# Visualize the path of the trace
fig, ax = plt.subplots(figsize = (11,6))
ax.scatter(recording.data['longitude'].values, recording.data['latitude'].values, zorder=1, alpha= 0.9, c='b', s=10)
ax.set_title('Visualization of the path')

if plot_map_zurich:
    # Bounding coordinates for map
    BBox = (8.5373,8.5593,
            47.3673, 47.3880)
    # Load map of Zurich
    ruh_m = plt.imread('/kaggle/input/24-exercise2/mapzurich.png')
    ax.set_title('GPS visualization in Zurich')
    ax.set_xlim(BBox[0],BBox[1])
    ax.set_ylim(BBox[2],BBox[3])
    ax.imshow(ruh_m, zorder=0, extent = BBox)

# Generate computed traces

Small example that adds an accelerometer magnitude trace

<code>Trace.from_modified(cls, title: str, values: np.ndarray, timestamps: np.ndarray, time_offset: float = 0)</code> can be used to save a new or modified trace for the Recording object.

<code>title</code>: Title for this collection of data.<br>
<code>values</code>: modified values<br>
<code>timestamps</code>: array of timestamps, one timestamp per value, usually directly taken from the original data<br>
<code>offset</code>: optional offset in case the "raw_timestamps" should match the original data<br>

Note: This change is only executed locally and not saved to the pkl file. Everytime you reload your dataset, you have to re-execute this steps. Alternatively, you can save a copy of your modified Recording using <code>save_to_pkl</code> (please be careful not to introduce errors in your origninal training set).


In [ ]:
# Trace must contain lilygo accelerometer data
assert('ax' in recording.data and 'ay' in recording.data and 'az' in recording.data)
# And those traces must be of the same length
assert(len(recording.data['ax'].values) == len(recording.data['ay'].values)  and len(recording.data['ax'].values) == len(recording.data['az'].values) )

magn = (recording.data['ax'].values**2 + recording.data['ay'].values**2 + recording.data['az'].values**2)**0.5

recording.data['a_mag'] = Trace.from_modified('Accelerometer magnitude', magn, recording.data['ax'].timestamps, recording.data['ax'].raw_timestamps[0][1])

recording.plot([['ax', 'ay', 'az'], ['a_mag']])

# FYI: How Timestamps Are Handled (You Probably Don't Need This)

Every time the smartphone receives a package via Bluetooth from the LilyGo smartwatch, up to 16 sensor values are stored at once, together with the current timestamp. Therefore, not every sensor reading has a perfectly defined timestamp that is 100% accurate.

Each <code>Trace</code> contains a list of <code>raw_timestamps</code>, which are tuples of the type `(trace index, timestamp)`.

The timestamps are standard Unix timestamps in milliseconds.

For example, the first two entries indicate "The 0th value arrived at 1645184293057, the 16th value at 1645184293144."

In [ ]:
recording.data["ax"].raw_timestamps

In [ ]:
recording.data['ax'].raw_timestamps[-1][1]-recording.data['ax'].raw_timestamps[0][1]

The LilyGo uses a FIFO with a constant sampling rate, so it should be a fair assumption, that the delay between two consecutive readings is always the same.

By default the <code>Trace.timestamps</code> array is linearly interpolated between the first and the last timestamp (<code>linspace(0, last_timestamp, num=len(values))</code>)

In [ ]:
# Compare "ideal" interpolated timestamps against actual received timestamps
recording.data['ideal_ts'] = Trace('ideal_ts', 
                                   [recording.data['timestamp'].values[0], recording.data['timestamp'].values[-1]], 
                                   [(0,recording.data['timestamp'].values[0]), 
                                    (len(recording.data['timestamp'].values), recording.data['timestamp'].values[-1])])
recording.plot(['timestamp', 'ideal_ts'], labels=['Received timestamps', 'Interpolated Timestamps'], ylabels=['ms'])